# ViewIt AI

In [1]:
import os
import openai
import pandas as pd
from prompts import *
from langchain import LLMChain
from langchain.tools import GooglePlacesTool
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import ZeroShotAgent, AgentExecutor

In [2]:

def load_data(filename) -> pd.DataFrame:
    df = pd.read_csv(f"data/{filename}")

    df['Date'] = pd.to_datetime(df['Date'])
    # .dt.date
    return df



def create_pandas_dataframe_agent(
        llm,
        df: pd.DataFrame,
        prefix: str,
        suffix: str,
        format_instructions: str,
        verbose: bool,
        **kwargs) -> AgentExecutor:
    """Construct a pandas agent from an LLM and dataframe."""

    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"Expected pandas object, got {type(df)}")

    input_variables = ["df", "input", "chat_history", "agent_scratchpad"]

    # Set up memory
    memory = ConversationBufferMemory(memory_key="chat_history")

    tools = [PythonAstREPLTool(locals={"df": df}), GooglePlacesTool()]

    prompt = ZeroShotAgent.create_prompt(
        tools=tools,
        prefix=prefix,
        suffix=suffix,
        format_instructions=format_instructions,
        input_variables=input_variables
    )
    partial_prompt = prompt.partial(df=str(df.head()))

    llm_chain = LLMChain(
        llm=llm,
        prompt=partial_prompt
    )
    tool_names = [tool.name for tool in tools]

    agent = ZeroShotAgent(llm_chain=llm_chain,
                          allowed_tools=tool_names, verbose=verbose)

    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=verbose,
        memory=memory,
        **kwargs
    )


In [3]:

# VARIABLES
MODEL_NAME = "gpt-4"
TEMPERATURE = 0.1
df = load_data('reidin_new.csv')
api_key = os.environ["OPENAI_API_KEY"]
org = ""

llm = ChatOpenAI(temperature=TEMPERATURE,
                    model_name='gpt-4',
                    openai_api_key=api_key)

# llm = OpenAI(temperature=TEMPERATURE,
#                 model_name=MODEL_NAME,
#                 openai_api_key=api_key)

# ViewIt OpenAI API key
openai.organization = org
openai.api_key = api_key

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    prefix=REIDIN_PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    verbose=True,
    handle_parsing_errors=True,
    # max_execution_time=30,
)

C:\Users\ga201\AppData\Local\Temp\ipykernel_22496\653537657.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


In [4]:
def ask(query, token_verbose = True):
    if token_verbose:
        with get_openai_callback() as cb:
            agent.run(query)
            print(cb)
    else:
        agent.run(query)

In [5]:
ask('What is the most expensive unit?')



> Entering new AgentExecutor chain...
The client wants to know about the most expensive property unit. I can find this by sorting the dataframe by the 'Price' column in descending order and then selecting the first row.
Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns',None)
df.sort_values(by='Price', ascending=False).head(1)
```
Observation:       Sales Type       Date  \
33852      Ready 2022-10-13   

                                                Location Property Type  \
33852  Signature Villas Frond N (Al Naghal), Palm Jum...         Villa   

       Bedrooms  Balcony Area  Built-up Area  Plot Size        Price  \
33852         6           0.0        26910.0   59569.38  600000000.0   

          Developer  Studio  
33852  Nakheel PJSC   False  
Thought:I now know the final answer.
Final Answer: The most expensive unit is a ready-to-move-in Villa located in Signature Villas Frond N (Al Naghal), Palm Jumeirah. It was sold on

In [13]:
agent.run('Search for the property type `villa` in JVC')



> Entering new AgentExecutor chain...
Yes
Action: python_repl_ast
Action Input: df[(df['Location'].str.contains('JVC')) & (df['Property Type'] == 'Villa')].sort_values(by='Price', ascending=False).head(1)
Observation:        Sales Type       Date  \
118584   Off Plan 2019-12-24   

                                                 Location Property Type  \
118584  JVC Villa and Townhouses - District 16, Jumeir...         Villa   

        Bedrooms  Balcony Area  Built-up Area  Plot Size       Price  \
118584         3           0.0         3828.0   27427.96  10354800.0   

           Developer  Studio  
118584  Nakheel PJSC   False  
Thought:I now know the final answer
Final Answer: The best villa to buy in JVC, based on the highest price, is a villa located in 'JVC Villa and Townhouses - District 16, Jumeirah Village Circle'. This villa is an off-plan property sold on December 24, 2019. It has 3 bedrooms, a built-up area of 3,828 square feet, and a plot size of 27,427.96 square feet.

"The best villa to buy in JVC, based on the highest price, is a villa located in 'JVC Villa and Townhouses - District 16, Jumeirah Village Circle'. This villa is an off-plan property sold on December 24, 2019. It has 3 bedrooms, a built-up area of 3,828 square feet, and a plot size of 27,427.96 square feet. The price of this villa is 10,354,800 AED and it was developed by Nakheel PJSC."

In [12]:
agent.run('What is the best villa to buy in JVC?')



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


The customer is asking for the best villa to buy in JVC. To determine the "best" villa, I will consider the villa with the highest price as the best. This is because in real estate, price is often a reflection of quality, location, size, and other desirable features. I will use the pandas dataframe to filter for villas in JVC and then sort by price in descending order.
Action: python_repl_ast
Action Input: df[(df['Location'].str.contains('JVC')) & (df['Property Type'] == 'Villa')].sort_values(by='Price', ascending=False).head(1)
Observation:        Sales Type       Date  \
118584   Off Plan 2019-12-24   

                                                 Location Property Type  \
118584  JVC Villa and Townhouses - District 16, Jumeir...         Villa   

        Bedrooms  Balcony Area  Built-up Area  Plot Size       Price  \
118584         3           0.0         3828.0   27427.96  10354800.0   

           Developer  Studio  
118584  Nakheel PJSC   False  
Thought:I now know the final 

"The best villa to buy in JVC, based on the highest price, is a villa located in 'JVC Villa and Townhouses - District 16, Jumeirah Village Circle'. This villa is an off-plan property sold on December 24, 2019. It has 3 bedrooms, a built-up area of 3,828 square feet, and a plot size of 27,427.96 square feet. The price of this villa is 10,354,800 AED and it was developed by Nakheel PJSC."